In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from sqlalchemy import create_engine
import sqlalchemy
from datetime import datetime

# Datos de conexión
account_url = "https://hackathonperiferiait.blob.core.windows.net"
container_name = "hackathon-data-source"
sas_token = "sp=rl&st=2024-09-17T22:37:26Z&se=2024-10-04T06:37:26Z&spr=https&sv=2022-11-02&sr=c&sig=CYqWUat230DmfqyGFxs32vGff5LkkPFN8WsEIwfRkUM%3D"

# Crear un cliente de BlobService
blob_service_client = BlobServiceClient(account_url=account_url, credential=sas_token)

# Obtener el cliente del contenedor
container_client = blob_service_client.get_container_client(container_name)

# Listar los blobs (archivos) en el contenedor
blobs_list = container_client.list_blobs()

# Descargar cada blob (archivo)
for blob in blobs_list:
    # Crear un cliente de blob para cada archivo
    blob_client = container_client.get_blob_client(blob)
    
    # Descargar el contenido
    with open(blob.name, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    
    print(f"Descargado: {blob.name}")

# Leer los archivos .parquet
df_job_applications = pd.read_parquet("Job_Applications.parquet")
df_jobs = pd.read_parquet("Jobs.parquet")


Descargado: Job_Applications.parquet
Descargado: Jobs.parquet


In [2]:
#print(df_job_applications.dtypes)
display(df_job_applications)

,job-app-id,job-app-created-at,job-app-rejected-at,job-app-sourced-application,candidate-id,job-id,reject-reason-id,candidate-connected,candidate-internal,candidate-referred,candidate-sourced,candidate-unsubscribed,candidate-tags,reject-reason,rejected-by-company,location-id,location-country,anonymized_resume
0,105172933,2024-08-01T00:27:10.291-05:00,2024-08-12T09:02:04.168-05:00,False,75277289,4785672,918805,True,False,False,False,False,[],No aplica para la vacante,True,276414.0,Colombia,\n[NOMBRE] \ncontacto: [TELÉFONO] \ncorreo:...
1,105172943,2024-08-01T00:27:25.608-05:00,2024-08-01T08:29:21.451-05:00,False,75277289,4758308,882625,True,False,False,False,False,[],Hoja de vida,True,276414.0,Colombia,\n[NOMBRE] \ncontacto: [TELÉFONO] \ncorreo:...
2,105173073,2024-08-01T00:29:36.914-05:00,2024-08-02T16:49:17.222-05:00,False,91154025,4784527,897254,False,False,False,False,False,[],Ubicación del candidato,True,276414.0,Colombia,\n \n \n \n \n \n \n \n \n \n \n \n \n \n} \...
3,105182504,2024-08-01T01:46:02.308-05:00,2024-08-01T08:09:53.617-05:00,False,91157990,4780778,897254,False,False,False,False,False,[],Ubicación del candidato,True,590109.0,Colombia,None
4,105183932,2024-08-01T02:02:24.173-05:00,2024-08-01T08:09:38.334-05:00,False,82445460,4780778,882625,True,False,False,False,False,['Proyectos'],Hoja de vida,True,276414.0,Colombia,[NOMBRE] \nname: [NOMBRE] \nemail: [EMAIL] \n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5339,111338356,2024-09-10T18:11:50.243-05:00,2024-09-11T18:35:05.938-05:00,False,47707799,4629826,882625,True,True,False,False,False,['Requerimientos'],Hoja de vida,True,277069.0,Colombia,hoja de vida \n \ninformación personal \n \n...
5340,111342325,2024-09-10T19:24:02.486-05:00,2024-09-11T10:31:04.516-05:00,False,96722889,4813708,882625,False,False,False,False,False,[],Hoja de vida,True,276414.0,Colombia,[NOMBRE] urrego \n \ningeniero de sistema s \...
5341,111344741,2024-09-10T20:21:12.896-05:00,2024-09-11T18:35:06.073-05:00,False,89210565,4629826,882625,True,False,False,False,False,[],Hoja de vida,True,276414.0,Colombia,j | a \n \nj o r g e c a m i lo \na n d r a [N...
5342,111345184,2024-09-10T20:31:59.129-05:00,2024-09-11T18:35:05.959-05:00,False,96725084,4629826,882625,False,False,False,False,False,[],Hoja de vida,True,276414.0,Colombia,"[NOMBRE] - cspo\nprofile\nresults, service-ori..."


In [3]:
# Definir el límite de caracteres
MAX_CHARS = 32768

# Truncar la columna 'anonymized_resume' antes de insertar
df_job_applications['anonymized_resume'] = df_job_applications['anonymized_resume'].apply(lambda x: x[:MAX_CHARS] if isinstance(x, str) else x)


df_job_applications['job-app-created-at'] = pd.to_datetime(df_job_applications['job-app-created-at'])
df_job_applications['job-app-rejected-at'] = pd.to_datetime(df_job_applications['job-app-rejected-at'])


df_job_applications['job-app-created-at'] = df_job_applications['job-app-created-at'].dt.strftime("%d/%m/%Y %H:%M:%S")
df_job_applications['job-app-rejected-at'] = df_job_applications['job-app-rejected-at'].dt.strftime("%d/%m/%Y %H:%M:%S")

In [5]:
# Conexión a MySQL
user = 'root'       # Reemplaza con tu usuario
password = 'Camilo12*'   # Reemplaza con tu contraseña
host = 'localhost'           # Por ejemplo, 'localhost' o la IP de tu servidor MySQL
database = 'sol_periferia'   # El nombre de tu base de datos

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Cargar los DataFrames en MySQL
df_job_applications.to_sql('job_applications', con=engine, if_exists='replace', index=False)
df_jobs.to_sql('jobs', con=engine, if_exists='replace', index=False)

print("Datos cargados en MySQL con éxito.")

Datos cargados en MySQL con éxito.
